In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
#baseline ver
VER = 6

import pandas as pd, numpy as np
from tqdm.notebook import tqdm
import os, sys, pickle, glob, gc
from collections import Counter
#import cudf, 
import itertools
#print('We will use RAPIDS version',cudf.__version__)

# test data(valid data)

In [3]:
INPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/input/'
OUTPUT_DIR = '/content/drive/MyDrive/kaggle/2022/OTTO/output/cris_baseline/'

In [4]:
#newitem_candidate
pred_df = pd.read_csv('/content/drive/MyDrive/kaggle/2022/OTTO/output/candidate/test_days_click_cart_order_popular_aids_preds.csv')
pred_df

,session,labels
0,11098528,108125 1603001 29735 166037 558573 1505419 158...
1,11098529,108125 1603001 29735 166037 558573 1505419 158...
2,11098530,108125 1603001 29735 166037 558573 1505419 158...
3,11098531,108125 1603001 29735 166037 558573 1505419 158...
4,11098532,108125 1603001 29735 166037 558573 1505419 158...
...,...,...
1801246,12899774,1460571 108125 184976 1844379 1006198 1531805 ...
1801247,12899775,1460571 108125 184976 1844379 1006198 1531805 ...
1801248,12899776,1460571 108125 184976 1844379 1006198 1531805 ...
1801249,12899777,1460571 108125 184976 1844379 1006198 1531805 ...


In [5]:
#test_labels
test_labels = pd.read_parquet('/content/drive/MyDrive/kaggle/2022/OTTO/input/cris_baseline/otto-validation/test_labels.parquet')
test_labels

,session,type,ground_truth
0,11098528,clicks,[1679529]
1,11098528,carts,[1199737]
2,11098528,orders,"[990658, 950341, 1462506, 1561739, 907564, 369..."
3,11098529,clicks,[1105029]
4,11098530,orders,[409236]
...,...,...,...
2212687,12899774,clicks,[1399483]
2212688,12899775,clicks,[1760714]
2212689,12899776,clicks,[1737908]
2212690,12899777,clicks,[384045]


# recall

In [7]:
%%time

#clicks,cart,orderのracall計算
def recall_k_evaluate(test_labels, pred_df, session_types, top_k):
    sub = pred_df.copy()
    #print(sub)
    sub.labels = sub.labels.apply(lambda x: [int(i) for i in x.split(' ')][:top_k])
    test_labels = test_labels.loc[test_labels['type']==session_types]
    test_labels = test_labels.merge(sub, how='left', on=['session'])
    test_labels['hits'] = test_labels.apply(lambda df: len(set(df.ground_truth).intersection(set(df.labels))), axis=1) 
    test_labels['gt_count'] = test_labels.ground_truth.str.len().clip(0,top_k)
    recall = test_labels['hits'].sum() / test_labels['gt_count'].sum()
    print(f'{session_types} recall =',recall)

#clicksの場合
#test label(CV変わるときのため), pred_df_clicks, type, top_kは変わるので引数にする。
recall_k_evaluate(test_labels, pred_df, 'clicks', 20)

clicks recall = 0.021153677456545984
CPU times: user 53.8 s, sys: 1.46 s, total: 55.3 s
Wall time: 55.1 s
